<a href="https://colab.research.google.com/github/hongsy0113/cose461-NLP-project-choi-hong/blob/main/sns_text_classification_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json
from pandas import json_normalize


In [2]:
# json data 파일들을 google drive에 저장
# google drive 에 있는 파일들을 접근하기 위해 mount
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Convert json file to dataFrame
# input file parameter should be str, path and name of json file
# output dataFrame contains utterances. 
# each utterance has dialogueInfo and participantsInfo (i.e gender)

def json2df (file):
  with open(file, 'r') as f:
    jdata = json.load(f)
  
  meta_list = [['header', 'dialogueInfo', 'numberOfParticipants'],
             ['header', 'dialogueInfo', 'numberOfUtterances'],
             ['header', 'dialogueInfo', 'numberOfTurns'],
             ['header', 'dialogueInfo', 'type'],
             ['header', 'dialogueInfo', 'topic'],
             ['header', 'dialogueInfo', 'dialogueID'],
            ]
  # dataframe of utterances 
  df1 = pd.json_normalize(jdata['data'][:], record_path=['body'], meta= meta_list, errors='ignore')
  # dataframe of participants info
  df2 = pd.json_normalize(jdata['data'], record_path=['header', 'participantsInfo'],  meta= [['header', 'dialogueInfo', 'dialogueID']], errors='ignore')
  # join df1 and df2 where (df1.dialogueID == df2.dialogueID and df1.participantID == df2.participantID)
  df = pd.merge(left = df1 , right = df2, how = "left", on = ["header.dialogueInfo.dialogueID", 'participantID' ],sort=False)

  # rename column names
  # 열이름 너무 길어서 바꿈. P는 참가자, U는 메시지, D는 대화, T는 turn 으로 통일시킴. 맘에 안 들면 바꿔도 ok
  df.rename(columns = {'header.dialogueInfo.numberOfParticipants' : 'P_num', 
                       'header.dialogueInfo.numberOfUtterances': 'U_num', 
                       'header.dialogueInfo.numberOfTurns':'T_num',
                       'header.dialogueInfo.type': 'D_type',
                       'header.dialogueInfo.topic': 'D_topic',
                       'header.dialogueInfo.dialogueID':'D_id',
                       'utteranceID':'U_id',
                       'participantID':'P_id',
                       'turnID':'T_id',
                       'date':'U_date',
                       'gender':'P_gender',
                       'residentialProvince':'P_residential',
                       'age':'P_age'}, inplace = True)
  # 대화 id, utterance id 를 index로 함. 마찬가지로 수정해도 됨.
  df.set_index(['D_id','U_id'], inplace=True)
  return df


In [4]:
# 행사.json, 식음료.json 등 여러 데이터셋을 하나로 만들기 위한 함수
def concat_json2df (file_list):
  result = None

  for i, file in enumerate(file_list):
    df = json2df(file)
    if (i==0):
      result = df.copy()
    else:
      result = pd.concat([result,df])
  return result

In [5]:
f_list = ['/content/gdrive/MyDrive/cose461/행사.json',
          '/content/gdrive/MyDrive/cose461/식음료.json']

test3 = concat_json2df(f_list)

test3.describe()

,utterance,P_id,U_date,T_id,time,P_num,U_num,T_num,D_type,D_topic,P_age,P_residential,P_gender
count,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752
unique,2543121,9,2873,20,1440,8,23,17,2,2,9,18,2
top,#@시스템#사진#,P01,2020-12-21,T3,17:55:00,2,20,8,일상 대화,식음료,20대,서울특별시,여성
freq,38365,1759889,8714,455462,5287,2981713,499120,599571,3380905,1821031,2470129,969946,2713080


In [ ]:
#test test test
# 이미 구현되어 있는 형태소 분석 툴을 써야 할 텐데 
# '괜찮을거가태' 처럼 표준형태와 다른 형태의 단어들이 우리 모델 학습에 중요할 거 같은데
# 이런 거 어케할지. 그냥 전처리 없이 raw단어로 할지

In [35]:
# Naive-Bayes Method.
test = test3[['utterance', 'P_gender']]
test['P_gender'].value_counts()

여성    2713080
남성     668672
Name: P_gender, dtype: int64

In [36]:
test[['P_gender', 'utterance']]

P_gender                   utterance
D_id                                 U_id                                     
223740cf-606e-5149-820c-7dce69513da1 U1         여성                   8월 말도 ㄱㅊ해
                                     U2         여성                그때도 글ㅋㅔ 안추울걸
                                     U3         여성                    29일로 할까?
                                     U4         여성  날짜라도 일단 확정하면 ㅋㅋㅋ 찾기 수월하니까~
                                     U5         여성                응응 그럼 29일로가자
...                                            ...                         ...
fd4c483f-28a3-5703-82d3-2e3b1a984078 U12        여성                        리조또랑
                                     U13        여성                           웅
                                     U14        여성                         수육에
                                     U15        여성                          헤헤
                                     U16        여성                        장난아냐

[3381752 rows x 2 columns]

In [37]:
# Gender to discrete variable
# 0 : '여성', 1 : '남성'
test['P_gender'] = [0 if gender=='여성' else 1 for gender in test['P_gender']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
!pip install soynlp
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy
!JAVA_HOME="C:\Program Files\Java\jdk-16.0.1"

     |████████████████████████████████| 416 kB 6.8 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.0 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates 

In [30]:
from soynlp.normalizer import *
from konlpy.tag import Okt
import re

okt = Okt()

def clean(doc) :

  doc = okt.pos(doc, norm=True, stem=True)
  new_doc = list()

  stop_tags = stop_tags = ['Determiner', 'Modifier', 'Josa', 'Foreign']
  for text, tag in doc:
    
    if tag in stop_tags :
      continue

    text = re.sub(r'[a-zA-Z0-9]+', '', text)  # remove digits.
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text) # 특수문자 제거
    text = emoticon_normalize(text, num_repeats=3) # remove repeated emoticon. e.g) ㅋㅋㅋㅋ=>ㅋㅋ, ㅠㅠㅠㅠ=>ㅠㅠ
    text = repeat_normalize(text, num_repeats=3) # remove repeated character.
    
    if tag in 'KoreanParticle' :
      text = ''.join(dict.fromkeys(text))

    new_doc.append(text)

  new_doc = ' '.join(new_doc)
  return new_doc 

In [29]:
okt.pos('이러지마 제바 ㄹㅇㄴㄹㄴㅇㄹㄴㅇㄹ ㅈㄷㄱ ㅋㅋ', norm=True, stem=True)

[('이르다', 'Verb'),
 ('제바', 'Noun'),
 ('ㄹㅇㄴㄹㄴㅇㄹㄴㅇㄹ', 'KoreanParticle'),
 ('ㅈㄷㄱ', 'KoreanParticle'),
 ('ㅋㅋ', 'KoreanParticle')]

In [38]:
test_women = test[test['P_gender']==0]
test_men = test[test['P_gender']==1]

print("Number of women : {}, Number of men : {}".format(test_women['P_gender'].value_counts()[0], test_men['P_gender'].value_counts()))

Number of women : 2713080, Number of men : 1    668672
Name: P_gender, dtype: int64


In [39]:
test_women = test_women.sample(n=668672, random_state=7)
test = pd.concat([test_women, test_men], sort=True)
test

P_gender   utterance
D_id                                 U_id                      
8633b7f8-0cb5-56aa-8485-b2014747162f U16          0        않나??
c17278aa-7319-5fbb-a728-935f1da985ed U4           0           ㅋ
c5f2ece2-6130-5eb1-8ddc-77d6866b0c0b U3           0  뭔데ㅠㅠㅠ 크흡ㅠㅠ
acb0fac6-cde2-54dd-8a2f-62b13360286e U6           0      어디말하는거
5f539e55-a110-51ab-9c97-833f5e62df2f U17          0     맥주를마시려고
...                                             ...         ...
68161a7a-77e5-58b2-8861-ca4d27e73019 U7           1      예를들어??
                                     U8           1    블랑 이런거??
                                     U11          1          옹옹
                                     U15          1     앟 ㅏ을게쏭ㅇ
                                     U16          1   저런걸로 사놓을겡

[1337344 rows x 2 columns]

In [40]:
# test = test3[['utterance', 'P_gender']]
# test['P_gender'] = [0 if gender=='여성' else 1 for gender in test['P_gender']]
test['utterance'] = test['utterance'].apply(clean)
test

P_gender   utterance
D_id                                 U_id                      
8633b7f8-0cb5-56aa-8485-b2014747162f U16          0         않다 
c17278aa-7319-5fbb-a728-935f1da985ed U4           0           ㅋ
c5f2ece2-6130-5eb1-8ddc-77d6866b0c0b U3           0  데 ㅠ 크다 흡 ㅠ
acb0fac6-cde2-54dd-8a2f-62b13360286e U6           0       어디 말다
5f539e55-a110-51ab-9c97-833f5e62df2f U17          0       맥주 말다
...                                             ...         ...
68161a7a-77e5-58b2-8861-ca4d27e73019 U7           1       예 들다 
                                     U8           1      블 이렇다 
                                     U11          1          옹옹
                                     U15          1   앟 ㅏ 을게쏭 ㅇ
                                     U16          1    저러다 사다 겡

[1337344 rows x 2 columns]

In [41]:
X = test['utterance']
y = test['P_gender']

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

In [44]:
vectorizer = CountVectorizer(max_features=7777).fit(X_train)
X_train_countvector = vectorizer.transform(X_train)

feature_names = vectorizer.get_feature_names()
print(feature_names)

['__', '___', 'ㄱㄲ', 'ㄱㄴ', 'ㄱㄴㄲ', 'ㄱㄷ', 'ㄱㅅ', 'ㄱㅇㄷ', 'ㄱㅊ', 'ㄱㅋ', 'ㄴㅋ', 'ㄴㅏ', 'ㄴㅐ', 'ㄷㅎ', 'ㄷㅏ', 'ㄷㅔ', 'ㄹㅇ', 'ㄹㅎ', 'ㅁㄹ', 'ㅁㅈ', 'ㅁㅊ', 'ㅁㅊㅋ', 'ㅁㅎ', 'ㅂㄷ', 'ㅂㄹ', 'ㅅㄱ', 'ㅅㅂ', 'ㅅㅇ', 'ㅇㄴ', 'ㅇㄷ', 'ㅇㄸ', 'ㅇㄹ', 'ㅇㅁ', 'ㅇㅂ', 'ㅇㅅ', 'ㅇㅈ', 'ㅇㅈㄹ', 'ㅇㅋ', 'ㅇㅋㄷ', 'ㅇㅎ', 'ㅇㅏ', 'ㅇㅓ', 'ㅇㅔ', 'ㅇㅣ', 'ㅈㄴ', 'ㅈㅅ', 'ㅊㅋ', 'ㅋㄱ', 'ㅋㄱㄲ', 'ㅋㄲ', 'ㅋㄲㄱ', 'ㅋㄴ', 'ㅋㄷ', 'ㅋㅁㅊ', 'ㅋㅅ', 'ㅋㅅㅂ', 'ㅋㅇ', 'ㅋㅈ', 'ㅋㅌ', 'ㅋㅎ', 'ㅋㅜ', 'ㅋㅠ', 'ㅌㅋ', 'ㅍㅎ', 'ㅎㄷ', 'ㅎㄹ', 'ㅎㅁ', 'ㅎㅅ', 'ㅎㅇ', 'ㅎㅋ', 'ㅎㅠ', 'ㅏㄴ', 'ㅏㄹ', 'ㅏㅇ', 'ㅐㄴ', 'ㅑㅇ', 'ㅓㄱ', 'ㅓㄴ', 'ㅓㄹ', 'ㅓㅅ', 'ㅓㅆ', 'ㅓㅇ', 'ㅔㅇ', 'ㅔㅋ', 'ㅕㄴ', 'ㅛㅓ', 'ㅜㄴ', 'ㅜㅅ', 'ㅜㅇ', 'ㅜㅋ', 'ㅜㅎ', 'ㅜㅠ', 'ㅜㅠㅋ', 'ㅜㅡ', 'ㅜㅡㅠ', 'ㅠㅁ', 'ㅠㅅ', 'ㅠㅋ', 'ㅠㅍ', 'ㅠㅎ', 'ㅠㅜ', 'ㅠㅜㅋ', 'ㅠㅡ', 'ㅡㄴ', 'ㅡㅁ', 'ㅡㅅ', 'ㅡㅇ', 'ㅡㅋ', 'ㅡㅜ', 'ㅡㅠ', 'ㅣㄴ', 'ㅣㅇ', '가가', '가게', '가격', '가공', '가구', '가기', '가까워지다', '가까이', '가깝다', '가꾸다', '가끔', '가나', '가내', '가노', '가누다', '가늘다', '가능', '가능성', '가능하다', '가다', '가다가', '가도', '가두', '가두다', '가드', '가득', '가득하다', '가든', '가디', '가디건', '가라', '가라아게', '가래', '가래떡', '가로', '가로수길', '가루', '가르다', '가르치다', '가리다', '가리비', '가마', '가만', '가만히', '가면', '가미', '가바', '가방',

In [43]:
# 전처리없는 모델.
classifier = MultinomialNB()
classifier.fit(X_train_countvector, y_train)

y_predict = classifier.predict(vectorizer.transform(X_val))
cm = confusion_matrix(y_val, y_predict)
df_cm = pd.DataFrame(cm, index = ['실제값(0)', '실제값(1)'], columns = ['예측값(0)', '예측값(1)'])
print(classification_report(y_val, y_predict))


              precision    recall  f1-score   support

           0       0.56      0.61      0.58    133504
           1       0.57      0.52      0.55    133965

    accuracy                           0.57    267469
   macro avg       0.57      0.57      0.57    267469
weighted avg       0.57      0.57      0.57    267469



In [21]:
classifier = MultinomialNB()
classifier.fit(X_train_countvector, y_train)

y_predict = classifier.predict(vectorizer.transform(X_val))
cm = confusion_matrix(y_val, y_predict)
df_cm = pd.DataFrame(cm, index = ['실제값(0)', '실제값(1)'], columns = ['예측값(0)', '예측값(1)'])
print(classification_report(y_val, y_predict))


              precision    recall  f1-score   support

           0       0.55      0.71      0.62    133504
           1       0.59      0.42      0.49    133965

    accuracy                           0.56    267469
   macro avg       0.57      0.56      0.55    267469
weighted avg       0.57      0.56      0.55    267469



In [ ]:
print(df_cm)

        예측값(0)  예측값(1)
실제값(0)  115617   96192
실제값(1)   78604  133212


In [20]:
from sklearn.svm import LinearSVC

model2 = LinearSVC()
model2.fit(X_train_countvector, y_train)

y_predict2 = model2.predict(vectorizer.transform(X_val))
cm = confusion_matrix(y_val, y_predict2)
df_cm = pd.DataFrame(cm, index = ['실제값(0)', '실제값(1)'], columns = ['예측값(0)', '예측값(1)'])
print(classification_report(y_val, y_predict2))



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


              precision    recall  f1-score   support

           0       0.59      0.44      0.50    133504
           1       0.55      0.69      0.61    133965

    accuracy                           0.57    267469
   macro avg       0.57      0.57      0.56    267469
weighted avg       0.57      0.57      0.56    267469

